# 50. データの入手・整形

News Aggregator Data Setをダウンロードし、以下の要領で学習データ（train.txt），検証データ（valid.txt），評価データ（test.txt）を作成せよ．  

ダウンロードしたzipファイルを解凍し，readme.txtの説明を読む．  
情報源（publisher）が”Reuters”, “Huffington Post”, “Businessweek”, “Contactmusic.com”, “Daily Mail”の事例（記事）のみを抽出する．  
抽出された事例をランダムに並び替える．  
抽出された事例の80%を学習データ，残りの10%ずつを検証データと評価データに分割し，それぞれtrain.txt，valid.txt，test.txtというファイル名で保存する．ファイルには，１行に１事例を書き出すこととし，カテゴリ名と記事見出しのタブ区切り形式とせよ（このファイルは後に問題70で再利用する）．  

学習データと評価データを作成したら，各カテゴリの事例数を確認せよ．  

In [38]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
#データを読み込む
df = pd.read_csv('newsCorpora.csv', header=None, sep='\t', names=['ID', 'TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP'])

#データを抽出する
df = df.loc[df['PUBLISHER'].isin(['Reuters', 'Huffington Post', 'Businessweek', 'Contactmusic.com', 'Daily Mail']), ['TITLE', 'CATEGORY']]


In [3]:
#データを分割する
#train : valid,test = 8:2
train, valid_test = train_test_split(df, test_size = 0.2, shuffle=True, random_state=0, stratify=df['CATEGORY'])
#valid,test を半分に分ける　8:1:1にする
valid, test = train_test_split(valid_test, test_size = 0.5, shuffle=True, random_state=0, stratify=valid_test['CATEGORY'])


In [4]:
#データを各ファイルに保存する
train.to_csv('./train.txt', sep='\t', index=False)
valid.to_csv('./valid.txt', sep='\t', index=False)
test.to_csv('./test.txt', sep='\t', index=False)

In [5]:
#各カテゴリの事例数を確認
print('train.txt')
print(train['CATEGORY'].value_counts())
print('valid.txt')
print(valid['CATEGORY'].value_counts())
print('test.txt')
print(test['CATEGORY'].value_counts())

train.txt
b    4502
e    4223
t    1219
m     728
Name: CATEGORY, dtype: int64
valid.txt
b    562
e    528
t    153
m     91
Name: CATEGORY, dtype: int64
test.txt
b    563
e    528
t    152
m     91
Name: CATEGORY, dtype: int64


# 51. 特徴量抽出

学習データ，検証データ，評価データから特徴量を抽出し，それぞれtrain.feature.txt，valid.feature.txt，test.feature.txtというファイル名で保存せよ． なお，カテゴリ分類に有用そうな特徴量は各自で自由に設計せよ．記事の見出しを単語列に変換したものが最低限のベースラインとなるであろう．

In [13]:
#前処理
import string
import re

def preprocessing(text):
    table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    text = text.translate(table)  # 記号をスペースに置換
    text = text.lower()  # 小文字化
    text = re.sub('[0-9]+', '0', text)  # 数字列を0に置換

    return text

In [14]:
# データの結合
df = pd.concat([train, valid, test], axis=0)

df.reset_index(drop=True, inplace=True)  

In [16]:
# 前処理
df['TITLE'] = df['TITLE'].map(lambda x: preprocessing(x))

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

# データの分割
train_valid = df[:len(train) + len(valid)]
test = df[len(train) + len(valid):]

vectorizer = TfidfVectorizer(min_df=10)　#出現数が少ない単語は省く

X_train_valid = vectorizer.fit_transform(train_valid['TITLE'])  
X_test = vectorizer.transform(test['TITLE']) #testは分けておく

X_train_valid = pd.DataFrame(X_train_valid.toarray(), columns=vectorizer.get_feature_names())
X_test = pd.DataFrame(X_test.toarray(), columns=vectorizer.get_feature_names())

# trainとvalidを分割
X_train = X_train_valid[:len(train)]
X_valid = X_train_valid[len(train):]

# データの保存
X_train.to_csv('train.feature.txt', sep='\t', index=False)
X_valid.to_csv('valid.feature.txt', sep='\t', index=False)
X_test.to_csv('test.feature.txt', sep='\t', index=False)

In [25]:
print(X_train.head())

    0d   0m  0million  0nd   0s  0st  0th   aa  aaliyah  abbvie  ...  yields  \
0  0.0  0.0       0.0  0.0  0.0  0.0  0.0  0.0      0.0     0.0  ...     0.0   
1  0.0  0.0       0.0  0.0  0.0  0.0  0.0  0.0      0.0     0.0  ...     0.0   
2  0.0  0.0       0.0  0.0  0.0  0.0  0.0  0.0      0.0     0.0  ...     0.0   
3  0.0  0.0       0.0  0.0  0.0  0.0  0.0  0.0      0.0     0.0  ...     0.0   
4  0.0  0.0       0.0  0.0  0.0  0.0  0.0  0.0      0.0     0.0  ...     0.0   

   york  you  young  your   yr  yuan  zac  zendaya  zone  
0   0.0  0.0    0.0   0.0  0.0   0.0  0.0      0.0   0.0  
1   0.0  0.0    0.0   0.0  0.0   0.0  0.0      0.0   0.0  
2   0.0  0.0    0.0   0.0  0.0   0.0  0.0      0.0   0.0  
3   0.0  0.0    0.0   0.0  0.0   0.0  0.0      0.0   0.0  
4   0.0  0.0    0.0   0.0  0.0   0.0  0.0      0.0   0.0  

[5 rows x 2139 columns]


# 52. 学習

51で構築した学習データを用いて，ロジスティック回帰モデルを学習せよ．

In [28]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=0,max_iter=1000)
model.fit(X_train, train['CATEGORY'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

# 53. 予測

52で学習したロジスティック回帰モデルを用い，与えられた記事見出しからカテゴリとその予測確率を計算するプログラムを実装せよ．

In [45]:
def predict_with_score(model, x):
    return [np.max(model.predict_proba(x), axis=1), model.predict(x)]

In [46]:
train_pred = predict_with_score(model, X_train)
test_pred = predict_with_score(model, X_test)

print(train_pred)

[array([0.38143102, 0.9162524 , 0.71503472, ..., 0.86423888, 0.97127036,
       0.73518722]), array(['b', 'b', 'e', ..., 'b', 'b', 'e'], dtype=object)]


# 54. 正解率の計測

52で学習したロジスティック回帰モデルの正解率を，学習データおよび評価データ上で計測せよ．

In [48]:
from sklearn.metrics import accuracy_score

train_accuracy = accuracy_score(train['CATEGORY'], train_pred[1])
test_accuracy = accuracy_score(test['CATEGORY'], test_pred[1])

print('学習データの正解率：' + str(train_accuracy))
print('評価データの正解率：' + str(test_accuracy))

学習データの正解率：0.9258808095952024
評価データの正解率：0.8755622188905547


# 55. 混同行列の作成

52で学習したロジスティック回帰モデルの混同行列（confusion matrix）を，学習データおよび評価データ上で作成せよ．

In [49]:
from sklearn.metrics import confusion_matrix

train_cm = confusion_matrix(train['CATEGORY'], train_pred[1])
print(train_cm)

[[4341   96    9   56]
 [  55 4155    2   11]
 [  86  130  498   14]
 [ 183  141    8  887]]
